In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .appName('Read and Write') \
                    .config("spark.jars", "/opt/scala-project_2.12-0.1.0-SNAPSHOT.jar") \
                    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark

2023-04-29 00:01:09,829 WARN util.Utils: Your hostname, localhost resolves to a loopback address: 127.0.0.1; using 172.18.0.2 instead (on interface eth0)
2023-04-29 00:01:09,830 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
2023-04-29 00:01:10,305 INFO spark.SparkContext: Running Spark version 3.4.0
2023-04-29 00:01:10,329 INFO resource.ResourceUtils: ==============================================================
2023-04-29 00:01:10,329 INFO resource.ResourceUtils: No custom resources configured for spark.driver.
2023-04-29 00:01:10,329 INFO resource.ResourceUtils: ==============================================================
2023-04-29 00:01:10,330 INFO spark.SparkContext: Submitted application: Read and Write
2023-04-29 00:01:10,344 INFO resource.ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, 

In [2]:
spark.version

'3.4.0'

## Create DataFrames

In [3]:
from pyspark.sql.types import Row, IntegerType
from pyspark.sql.functions import col

row_list_data = [Row('Alice', 1), Row('Braga', 2), Row('Steve', 3)]
df = spark.createDataFrame(row_list_data, ['name', 'id'])

df = df.withColumn("id", col("id").cast(IntegerType()))
    
df.createOrReplaceTempView('users')

df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)



2023-04-29 00:01:21,900 INFO codegen.CodeGenerator: Code generated in 115.661801 ms
2023-04-29 00:01:21,931 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-04-29 00:01:21,940 INFO scheduler.DAGScheduler: Got job 0 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-04-29 00:01:21,941 INFO scheduler.DAGScheduler: Final stage: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0)
2023-04-29 00:01:21,941 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-04-29 00:01:21,942 INFO scheduler.DAGScheduler: Missing parents: List()
2023-04-29 00:01:21,944 INFO scheduler.DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[6] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-04-29 00:01:22,000 INFO memory.MemoryStore: Block broadcast_0 stored as values in memory (estimated size 12.6 KiB, free 434.4 MiB)
2023-04-29 00:01:22,019 INFO memory.MemoryStore: Block broadcast_0_

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



2023-04-29 00:01:22,969 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 918 ms on localhost (executor 1) (1/1)
2023-04-29 00:01:22,970 INFO cluster.YarnScheduler: Removed TaskSet 0.0, whose tasks have all completed, from pool 
2023-04-29 00:01:22,974 INFO python.PythonAccumulatorV2: Connected to AccumulatorServer at host: 127.0.0.1 port: 42847
2023-04-29 00:01:22,976 INFO scheduler.DAGScheduler: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0) finished in 1.023 s
2023-04-29 00:01:22,978 INFO scheduler.DAGScheduler: Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
2023-04-29 00:01:22,978 INFO cluster.YarnScheduler: Killing all running tasks in stage 0: Stage finished
2023-04-29 00:01:22,979 INFO scheduler.DAGScheduler: Job 0 finished: showString at NativeMethodAccessorImpl.java:0, took 1.048188 s
2023-04-29 00:01:22,989 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-04-29 00:

## Access via SparkSQL

In [4]:
from pyspark.sql.types import StringType

# register class that implements UDF interface
spark.udf.registerJavaFunction("lowerCaseString", "com.myudfs.LowerCaseString", returnType=StringType())

# call register function from class MathUDFs that will register the udf in spark 
spark.sparkContext._jvm.com.myudfs.MathUDFs.registerUdf()

JavaObject id=o70

In [5]:
spark.sql("""
    SELECT isGreaterThanZero(id), name
    FROM users
    """).show()

2023-04-29 00:01:23,294 INFO codegen.CodeGenerator: Code generated in 24.006056 ms
2023-04-29 00:01:23,317 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-04-29 00:01:23,318 INFO scheduler.DAGScheduler: Got job 2 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-04-29 00:01:23,318 INFO scheduler.DAGScheduler: Final stage: ResultStage 2 (showString at NativeMethodAccessorImpl.java:0)
2023-04-29 00:01:23,318 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-04-29 00:01:23,318 INFO scheduler.DAGScheduler: Missing parents: List()
2023-04-29 00:01:23,318 INFO scheduler.DAGScheduler: Submitting ResultStage 2 (MapPartitionsRDD[8] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-04-29 00:01:23,323 INFO memory.MemoryStore: Block broadcast_2 stored as values in memory (estimated size 25.9 KiB, free 434.3 MiB)
2023-04-29 00:01:23,325 INFO memory.MemoryStore: Block broadcast_2_p

+---------------------+-----+
|isGreaterThanZero(id)| name|
+---------------------+-----+
|                 true|Alice|
|                 true|Braga|
|                 true|Steve|
+---------------------+-----+



2023-04-29 00:01:24,183 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-04-29 00:01:24,184 INFO scheduler.DAGScheduler: Got job 3 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-04-29 00:01:24,184 INFO scheduler.DAGScheduler: Final stage: ResultStage 3 (showString at NativeMethodAccessorImpl.java:0)
2023-04-29 00:01:24,184 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-04-29 00:01:24,184 INFO scheduler.DAGScheduler: Missing parents: List()
2023-04-29 00:01:24,184 INFO scheduler.DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[8] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-04-29 00:01:24,187 INFO memory.MemoryStore: Block broadcast_3 stored as values in memory (estimated size 25.9 KiB, free 434.3 MiB)
2023-04-29 00:01:24,190 INFO memory.MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 12.6 KiB, free 434.3 MiB)
2023-04-2

In [6]:
spark.sql("""
    SELECT id, lowerCaseString(name)
    FROM users
    """).show()

2023-04-29 00:01:24,315 INFO codegen.CodeGenerator: Code generated in 12.17728 ms
2023-04-29 00:01:24,323 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-04-29 00:01:24,324 INFO scheduler.DAGScheduler: Got job 4 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-04-29 00:01:24,324 INFO scheduler.DAGScheduler: Final stage: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0)
2023-04-29 00:01:24,324 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-04-29 00:01:24,324 INFO scheduler.DAGScheduler: Missing parents: List()
2023-04-29 00:01:24,325 INFO scheduler.DAGScheduler: Submitting ResultStage 4 (MapPartitionsRDD[10] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-04-29 00:01:24,328 INFO memory.MemoryStore: Block broadcast_4 stored as values in memory (estimated size 14.1 KiB, free 434.3 MiB)
2023-04-29 00:01:24,330 INFO memory.MemoryStore: Block broadcast_4_p

+---+---------------------+
| id|lowerCaseString(name)|
+---+---------------------+
|  1|                alice|
|  2|                braga|
|  3|                steve|
+---+---------------------+



2023-04-29 00:01:24,407 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 4.0 (TID 4) in 75 ms on localhost (executor 1) (1/1)
2023-04-29 00:01:24,407 INFO cluster.YarnScheduler: Removed TaskSet 4.0, whose tasks have all completed, from pool 
2023-04-29 00:01:24,408 INFO scheduler.DAGScheduler: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0) finished in 0.082 s
2023-04-29 00:01:24,409 INFO scheduler.DAGScheduler: Job 4 is finished. Cancelling potential speculative or zombie tasks for this job
2023-04-29 00:01:24,409 INFO cluster.YarnScheduler: Killing all running tasks in stage 4: Stage finished
2023-04-29 00:01:24,409 INFO scheduler.DAGScheduler: Job 4 finished: showString at NativeMethodAccessorImpl.java:0, took 0.085547 s
2023-04-29 00:01:24,417 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-04-29 00:01:24,417 INFO scheduler.DAGScheduler: Got job 5 (showString at NativeMethodAccessorImpl.java:0) with 1 output partition

## Access via DataFrame API

In [7]:
from py4j.java_gateway import java_import
from pyspark.sql.column import Column, _to_java_column, _to_seq 

def greater_than_zero_udf(df_col: str):
    greaterThanZeroUDF = spark.sparkContext._jvm.com.myudfs.MathUDFs.isGreaterThanZeroUDF()
    return Column(greaterThanZeroUDF.apply(_to_seq(spark.sparkContext, [df_col], _to_java_column)))

def multiplier_udf(df_col: str, multiply_by: int):
    def multiplier(df_col: str):
        multiplierUDF = spark.sparkContext._jvm.com.myudfs.MathUDFs.multiplyBy(multiply_by)
        return Column(multiplierUDF.apply(_to_seq(spark.sparkContext, [df_col], _to_java_column)))

    return multiplier(df_col)

# work in progress
def lower_case_udf(df_col: str):
    lower_case_class = spark.sparkContext._jvm.com.myudfs.LowerCaseString
    return Column(lower_case_class.apply(_to_seq(spark.sparkContext, [df_col], _to_java_column)))

In [8]:
df \
    .withColumn("greater_than_zero", greater_than_zero_udf("id")) \
    .withColumn("multiplied", multiplier_udf("id", 10)) \
    .show()

2023-04-29 00:01:24,591 INFO codegen.CodeGenerator: Code generated in 12.876954 ms
2023-04-29 00:01:24,600 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-04-29 00:01:24,601 INFO scheduler.DAGScheduler: Got job 6 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-04-29 00:01:24,601 INFO scheduler.DAGScheduler: Final stage: ResultStage 6 (showString at NativeMethodAccessorImpl.java:0)
2023-04-29 00:01:24,601 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-04-29 00:01:24,602 INFO scheduler.DAGScheduler: Missing parents: List()
2023-04-29 00:01:24,602 INFO scheduler.DAGScheduler: Submitting ResultStage 6 (MapPartitionsRDD[12] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-04-29 00:01:24,606 INFO memory.MemoryStore: Block broadcast_6 stored as values in memory (estimated size 29.3 KiB, free 434.2 MiB)
2023-04-29 00:01:24,608 INFO memory.MemoryStore: Block broadcast_6_

+-----+---+-----------------+----------+
| name| id|greater_than_zero|multiplied|
+-----+---+-----------------+----------+
|Alice|  1|             true|        10|
|Braga|  2|             true|        20|
|Steve|  3|             true|        30|
+-----+---+-----------------+----------+



2023-04-29 00:01:26,485 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 7.0 (TID 7) in 1785 ms on localhost (executor 2) (1/1)
2023-04-29 00:01:26,485 INFO cluster.YarnScheduler: Removed TaskSet 7.0, whose tasks have all completed, from pool 
2023-04-29 00:01:26,486 INFO scheduler.DAGScheduler: ResultStage 7 (showString at NativeMethodAccessorImpl.java:0) finished in 1.793 s
2023-04-29 00:01:26,486 INFO scheduler.DAGScheduler: Job 7 is finished. Cancelling potential speculative or zombie tasks for this job
2023-04-29 00:01:26,486 INFO cluster.YarnScheduler: Killing all running tasks in stage 7: Stage finished
2023-04-29 00:01:26,486 INFO scheduler.DAGScheduler: Job 7 finished: showString at NativeMethodAccessorImpl.java:0, took 1.795666 s
2023-04-29 00:01:26,500 INFO codegen.CodeGenerator: Code generated in 9.150659 ms
